# Connecting to the Adzuna API and Gmaps API to gather data
* Gathering job postings with lat and lng data, and created date.
* Using the lat and lng to get the city, state, and country with the help of google's geocoding api.
* Small amount of formatting

In [2]:
#imports 
import json
import requests
import pandas as pd
import pprint as pprint
from pandas.io.json import json_normalize
import gmaps
import gmaps.datasets

### CSV file path variables

In [3]:
# data file to be saved and pulled from
rawData = "../data/rawData/master_normalized_raw_dataframe.csv"
# renamed data file to be saved and pulled from
orgRawData = "../data/rawData/master_organized_raw_dataframe.csv"
# No missing data file to be saved and pulled from
noMisingData = "../data/rawData/master_no_missing_raw_data.csv"
# Raw geo data file to be saved and pulled from
rawGeoData = "../data/rawData/master_geo_raw_dataframe.csv"

# path csv file location
pathRawData = "../data/rawData/"

### Calling Config file to store keys in variables

In [16]:
# Remember to update the config file with your API key
from config import api_key
from config import api_id
from config import gkey

### Connecting to Adzuna API

In [ ]:
url = "https://api.adzuna.com/v1/api/jobs/us/search/"
#have to create iteration to get all pages of data
page = "1"
api_details ="?" + "app_id=" + api_id + "&app_key=" + api_key
# Build query URL
query_url = url + page + api_details
query_url

In [ ]:
#checking response type
data_response = requests.get(query_url)
data_json = data_response.json()
type(data_json)


### Iterating through the API using json_normalize

In [ ]:
#Getting data using json_normalize
#LINK: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.io.json.json_normalize.html
#Page 33683 iteration complete
start_page = 21259
end_page = 50001
pages_test = [str(x) for x in range(start_page,end_page)]

numbers = start_page + 5000

counter = 0
# Build query URL
data_pages_df = pd.DataFrame()

for page_num in pages_test:
    try:
        response = requests.get(url + page_num + api_details).json()

        #create a df to store the normalized page that has the job postings
        normalized_page = json_normalize(response['results'])

        #add normalized data into a df
        data_page_df = pd.DataFrame(normalized_page)

        #add the url column to df incase an error occurs you know what page you were on
        data_page_df['query_url'] = 'page number ' + page_num

        #append to a new df so each page can be saved
        #Sorting because non-concatenation axis is not aligned
        data_pages_df = data_pages_df.append(data_page_df, sort=True)

        counter += 1
        print(f"Page {page_num} iteration complete")
    except ValueError:  # includes simplejson.decoder.JSONDecodeError
        print('JSONDecodeError occured. Skipping...')
        
    if counter == numbers:
        data_pages_df.to_csv(f"../data/rawData/master_normalized_raw_dataframe_{str(counter)}.csv", index=False)
        number += 5000
        
print(f'total rows iterated | {counter * 10}')   
      

In [ ]:
#save raw data to dataRaw folder
data_pages_df.to_csv(rawData, index=False)

In [21]:
#calling the csv raw data
csv_file = pd.read_csv(rawData)
csv_file.head(3)


,Unnamed: 0,__CLASS__,adref,category.__CLASS__,category.label,category.tag,company.__CLASS__,company.display_name,contract_time,contract_type,...,id,latitude,location.__CLASS__,location.area,location.display_name,longitude,query_url,redirect_url,salary_is_predicted,title
0,0,Adzuna::API::Response::Job,eyJhbGciOiJIUzI1NiJ9.eyJpIjoxMzEyODA1MDkzLCJzI...,Adzuna::API::Response::Category,Healthcare & Nursing Jobs,healthcare-nursing-jobs,Adzuna::API::Response::Company,CompHealth,NaN,NaN,...,1312805093,44.239491,Adzuna::API::Response::Location,"['US', 'New York', 'Jefferson County', 'Clayton']","Clayton, Jefferson County",-76.085776,page number 1,https://www.adzuna.com/land/ad/1312805093?se=b...,0,Physician: Family Physician Job Opening in Ups...
1,1,Adzuna::API::Response::Job,eyJhbGciOiJIUzI1NiJ9.eyJpIjoxMzEyODA1OTAxLCJzI...,Adzuna::API::Response::Category,Healthcare & Nursing Jobs,healthcare-nursing-jobs,Adzuna::API::Response::Company,CompHealth,NaN,NaN,...,1312805901,40.305232,Adzuna::API::Response::Location,"['US', 'Ohio']","Ohio, US",-82.824785,page number 1,https://www.adzuna.com/land/ad/1312805901?se=b...,0,Physician: Urology Need in Western Ohio with L...
2,2,Adzuna::API::Response::Job,eyJhbGciOiJIUzI1NiJ9.eyJzIjoiYkR5b01pMkhSQ2VYc...,Adzuna::API::Response::Category,Healthcare & Nursing Jobs,healthcare-nursing-jobs,Adzuna::API::Response::Company,CompHealth,NaN,NaN,...,1328241931,41.508367,Adzuna::API::Response::Location,"['US', 'Connecticut', 'New Haven County', 'Che...","Cheshire, New Haven County",-72.910620,page number 1,https://www.adzuna.com/land/ad/1328241931?se=b...,0,Physician: Internal Medicine Physician Job in ...


### Defining columns to use and renaming the columns

In [22]:
#making a copy
csv_file_copy = csv_file.copy()

#getting only the rows that we are using
csv_file_copy = csv_file_copy[["id","title","category.label","company.display_name", "location.display_name","created","latitude","longitude" ]]
csv_file_copy.head()

,id,title,category.label,company.display_name,location.display_name,created,latitude,longitude
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",2019-10-25T12:38:41Z,44.239491,-76.085776
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",2019-10-25T12:38:57Z,40.305232,-82.824785
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",2019-11-06T20:28:13Z,41.508367,-72.910620
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",2019-11-11T22:10:17Z,37.730605,-88.933126
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",2019-10-25T12:38:53Z,41.601859,-72.657316


In [27]:
#Renaming the column names

renamed_csv_file = csv_file_copy.rename(columns = {
    "id": "job posting id",
    "title": "job title",
    "category.label":"category",
    "company.display_name":"company name",
    "location.display_name":"location",
    "created":"date created",
    "latitude":"lat",
    "longitude":"lng"
})
renamed_csv_file.head()

,job posting id,job title,category,company name,location,date created,lat,lng
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",2019-10-25T12:38:41Z,44.239491,-76.085776
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",2019-10-25T12:38:57Z,40.305232,-82.824785
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",2019-11-06T20:28:13Z,41.508367,-72.910620
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",2019-11-11T22:10:17Z,37.730605,-88.933126
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",2019-10-25T12:38:53Z,41.601859,-72.657316


In [28]:
#save raw data to dataRaw folder
renamed_csv_file.to_csv(orgRawData, index=False)

In [29]:
#calling the csv org data
org_raw_csv_file = pd.read_csv(orgRawData)
org_raw_csv_file_copy = org_raw_csv_file.copy()
org_raw_csv_file_copy.head()

,job posting id,job title,category,company name,location,date created,lat,lng
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",2019-10-25T12:38:41Z,44.239491,-76.085776
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",2019-10-25T12:38:57Z,40.305232,-82.824785
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",2019-11-06T20:28:13Z,41.508367,-72.910620
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",2019-11-11T22:10:17Z,37.730605,-88.933126
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",2019-10-25T12:38:53Z,41.601859,-72.657316


### Checking the length of the dataframe and making changes to have each column the same size

In [30]:
#Count Rows of all columns
org_raw_csv_file_copy.count()

job posting id    1000
job title         1000
category          1000
company name      1000
location          1000
date created      1000
lat                983
lng                983
dtype: int64

In [31]:
#Delete Rows with missing data from lat and lng
org_data = org_raw_csv_file_copy.dropna()
org_data.count()

job posting id    983
job title         983
category          983
company name      983
location          983
date created      983
lat               983
lng               983
dtype: int64

In [32]:
#Save organized raw data into rawData folder
org_data.to_csv(noMisingData, index=False)

In [33]:
#calling the csv org data
no_missing_data_csv_file = pd.read_csv(noMisingData)
no_missing_data = no_missing_data_csv_file.copy()
no_missing_data.head()

,job posting id,job title,category,company name,location,date created,lat,lng
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",2019-10-25T12:38:41Z,44.239491,-76.085776
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",2019-10-25T12:38:57Z,40.305232,-82.824785
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",2019-11-06T20:28:13Z,41.508367,-72.910620
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",2019-11-11T22:10:17Z,37.730605,-88.933126
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",2019-10-25T12:38:53Z,41.601859,-72.657316


### Creating a new column with Lat and Lng combined

In [36]:
#Creating new column for combined LatLng
no_missing_data['latlng'] = no_missing_data['lat'].map(str) + "," + no_missing_data['lng'].map(str)

LatLng = list(no_missing_data['latlng'])
LatLng.head()

['44.239490999999994,-76.085776',
 '40.305232000000004,-82.82478499999998',
 '41.508367,-72.91062',
 '37.730605,-88.933126',
 '41.60185900000001,-72.65731600000002',
 '44.022931,-73.145015',
 '33.160641999999996,-117.238216',
 '40.621455,-79.152535',
 '47.515458,-111.28035700000001',
 '40.305232000000004,-82.82478499999998',
 '46.817181,-95.845325',
 '42.104245,-72.318952',
 '43.639036,-72.922874',
 '34.213396,-79.820616',
 '39.863792,-89.15483',
 '41.032294,-73.768378',
 '42.331666999999996,-89.15154799999998',
 '35.962317999999996,-79.97522099999998',
 '33.45,-112.07',
 '36.420403,-81.473438',
 '35.370461,-94.413065',
 '40.824806,-74.109314',
 '40.457812,-92.17129200000001',
 '40.288232,-74.617769',
 '42.522869,-75.57467700000002',
 '37.56,-77.45',
 '37.669746,-120.999103',
 '37.610182,-85.163472',
 '41.60185900000001,-72.65731600000002',
 '41.410849,-75.665902',
 '39.620378,-75.748778',
 '43.459514,-76.228818',
 '35.58754000000001,-88.859124',
 '36.154447999999995,-109.5526070000000

### Reverse geocoding request with Geocoding API using the LatLng column to get the the city, state, country. Storing each API url in new geocoding column to append to the empty columns of city, state, and country

In [ ]:
#Create new columns for the new data pulling from google api
no_missing_data['geocode_data'] = ''
no_missing_data['city'] = ''
no_missing_data['state'] = ''
no_missing_data['country'] = ''

#Create a function to call each url with the datas Lat Long
def reverse_geocode(latlng):
    #this going to store the url for each LatLng
    result = {}
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?latlng="
    api_geo_details ="&key=" + gkey
    query_geo_url = geocode_url + latlng + api_geo_details
    geo_data_response = requests.get(query_geo_url)
    geo_data_json = geo_data_response.json()
    
    if len(geo_data_json['results']) > 0:
        result = geo_data_json['results'][0]
    return result

#use map to call the function for all column LatLng
no_missing_data['geocode_data'] = no_missing_data['latlng'].map(reverse_geocode)
no_missing_data.head(3)

In [39]:
#Created a function that searches for Country
def parse_country(geocode_data):
    if (not geocode_data is None) and ('address_components' in geocode_data):
        for component in geocode_data['address_components']:
            if 'country' in component['types']:
                return component['long_name']
    return None

#Created a function that searches for state
def parse_state(geocode_data):
    if (not geocode_data is None) and ('address_components' in geocode_data):
        for component in geocode_data['address_components']:
            if 'administrative_area_level_1' in component['types']:
                return component['long_name']
    return None

#Created a function that searches for city/town 
def parse_city(geocode_data):
    if (not geocode_data is None) and ('address_components' in geocode_data):
        for component in geocode_data['address_components']:
            if 'locality' in component['types']:
                return component['long_name']
            elif 'postal_town' in component['types']:
                return component['long_name']
            elif 'administrative_area_level_2' in component['types']:
                return component['long_name']
            elif 'administrative_area_level_1' in component['types']:
                return component['long_name']
    return None

#append these functions for city state and country using the maps function
no_missing_data['city'] = no_missing_data['geocode_data'].map(parse_city)
no_missing_data['state'] = no_missing_data['geocode_data'].map(parse_state)
no_missing_data['country'] = no_missing_data['geocode_data'].map(parse_country)
no_missing_data.head()

,job posting id,job title,category,company name,location,date created,lat,lng,latlng,geocode_data,city,state,country
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",2019-10-25T12:38:41Z,44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",2019-10-25T12:38:57Z,40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",2019-11-06T20:28:13Z,41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",2019-11-11T22:10:17Z,37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",2019-10-25T12:38:53Z,41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States


### Saving the dataframe to a csv within the rawData folder. Next is the Clean_Up_Data notebook

In [41]:
#save raw data to dataRaw folder
geo_raw_df = no_missing_data
geo_raw_df.to_csv(rawGeoData, index=False)
geo_raw_df.head()

,job posting id,job title,category,company name,location,date created,lat,lng,latlng,geocode_data,city,state,country
0,1312805093,Physician: Family Physician Job Opening in Ups...,Healthcare & Nursing Jobs,CompHealth,"Clayton, Jefferson County",2019-10-25T12:38:41Z,44.239491,-76.085776,"44.239490999999994,-76.085776","{'address_components': [{'long_name': '512', '...",Clayton,New York,United States
1,1312805901,Physician: Urology Need in Western Ohio with L...,Healthcare & Nursing Jobs,CompHealth,"Ohio, US",2019-10-25T12:38:57Z,40.305232,-82.824785,"40.305232000000004,-82.82478499999998","{'address_components': [{'long_name': '5629', ...",Sunbury,Ohio,United States
2,1328241931,Physician: Internal Medicine Physician Job in ...,Healthcare & Nursing Jobs,CompHealth,"Cheshire, New Haven County",2019-11-06T20:28:13Z,41.508367,-72.910620,"41.508367,-72.91062",{'address_components': [{'long_name': 'Cheshir...,Cheshire,Connecticut,United States
3,1334502463,Physician: A Locums Otolaryngologist Is Wanted...,Healthcare & Nursing Jobs,CompHealth,"Marion, Williamson County",2019-11-11T22:10:17Z,37.730605,-88.933126,"37.730605,-88.933126","{'address_components': [{'long_name': '805', '...",Marion,Illinois,United States
4,1312805719,"Physician: Hospitalist needed in Hartford, CT ...",Healthcare & Nursing Jobs,CompHealth,"Connecticut, US",2019-10-25T12:38:53Z,41.601859,-72.657316,"41.60185900000001,-72.65731600000002","{'address_components': [{'long_name': '87', 's...",Cromwell,Connecticut,United States
